In [1]:
!conda install numpy netcdf4 --yes

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda:
#
netcdf4                   1.3.0                    py36_0    conda-forge
numpy                     1.13.3          py36_blas_openblas_200  [blas_openblas]  conda-forge


In [2]:
from netCDF4 import Dataset, num2date
import numpy as np

In [3]:
sites = {
    '1041Y5': { 'northing': [828536.45, 849341.47], 'easting': [79583.28, 95719.28] },
    '6Y2': { 'northing': [29575.83, 42777.92], 'easting': [135543.73, 144694.19] },
    '93Y1': { 'northing': [99151.28, 116807.98], 'easting': [560075.80, 578900.92] },
    '143X1': { 'northing': [139629.74, 159749.73], 'easting': [528000.69, 534396.56] },
    '1241X4': { 'northing': [321276.61, 331718.89], 'easting': [306848.73, 310933.03] },
    '783X4': { 'northing': [597326.35, 625399.10], 'easting': [238808.80, 259227.59] },
    '734U1': { 'northing': [549502.09, 559512.58], 'easting': [236777.56, 245708.56] },
    '1047Y2': { 'northing': [839294.59, 851032.62], 'easting': [191501.08, 225094.96] },
    '702Y1': { 'northing': [528737.62, 541455.26], 'easting': [229114.49, 253308.78] },
    '17X2': { 'northing': [59646.77, 72053.79], 'easting': [237064.92, 265252.37] }
}

In [4]:
parameter = 'tas'
timeUnits = 'days since 1961-01-01'

In [5]:
# Using an internal thredd sever
ncFile = Dataset('http://thredds-service:8080/thredds/dodsC/chess/tas/agg', 'r')

In [6]:
def findRegionOfInterest(boundry, eastings, northings):
    [eastingEast, eastingWest] = boundry['easting']
    [northingSouth, northingNorth] = boundry['northing']
    eastingInds = np.where((eastings > eastingEast) & (eastings < eastingWest))[0]
    northingInds = np.where((northings > northingSouth) & (northings < northingNorth))[0]
    return (northingInds, eastingInds)

def groupByMonth(ncTime, timeUnits):
    timeSeries = np.array([num2date(time, timeUnits) for time in ncTime], dtype='datetime64[M]')
    timeUnique = np.unique(timeSeries)
    return [ (time, np.where(timeSeries == time)[0]) for time in timeUnique]

def aggregateByTime(pararmeterData, timeByMonth, rois):
    [northingInds, eastingInds] = rois
    
    def createRow(timeInds):
        values = pararmeterData[timeInds, northingInds, eastingInds]
        return { 'mean': np.mean(values), 'min': np.min(values), 'max': np.max(values) }
    
    return [ (str(time), createRow(timeInds)) for time, timeInds in timeByMonth ]

def aggregateSites(sites, parameter, timeUnits, ncFile):
    eastings = ncFile.variables['x'][:]
    northings = ncFile.variables['y'][:]
    ncTime = ncFile.variables['time']
    pararmeterData = ncFile.variables[parameter]
    
    rois = { siteName: findRegionOfInterest(sites[siteName], eastings, northings) for siteName in sites }
    timeByMonth = groupByMonth(ncTime, timeUnits)
    
    return { site: aggregateByTime(pararmeterData, timeByMonth, rois[site])  for site in rois }

In [8]:
aggSites = aggregateSites(sites, parameter, timeUnits, ncFile)

In [9]:
# List minimum, maximum and mean temperatures (K) for each month at site '1041Y5'
aggSites['1041Y5']

[('1980-01', {'max': 279.58167, 'mean': 276.49743401759531, 'min': 269.99387}),
 ('1980-02', {'max': 281.48889, 'mean': 277.88409961685824, 'min': 269.18579}),
 ('1980-03', {'max': 280.24799, 'mean': 277.28812316715545, 'min': 270.27368}),
 ('1980-04', {'max': 285.29453, 'mean': 280.68249158249159, 'min': 276.27264}),
 ('1980-05', {'max': 289.353, 'mean': 283.54091316389702, 'min': 277.09433}),
 ('1980-06', {'max': 288.12076, 'mean': 284.54170875420874, 'min': 280.02722}),
 ('1980-07', {'max': 292.43561, 'mean': 285.34196806777453, 'min': 280.59891}),
 ('1980-08', {'max': 288.60669, 'mean': 286.18666503747147, 'min': 281.44046}),
 ('1980-09', {'max': 288.51794, 'mean': 285.37952441077442, 'min': 279.79068}),
 ('1980-10', {'max': 284.93933, 'mean': 281.08137829912022, 'min': 275.35486}),
 ('1980-11', {'max': 283.55228, 'mean': 279.08207070707073, 'min': 271.98422}),
 ('1980-12', {'max': 282.48984, 'mean': 278.47336265884655, 'min': 272.84097}),
 ('1981-01', {'max': 281.84814, 'mean': 27